In [1]:
import pickle
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
import multiprocessing
from copy import deepcopy
from gensim.models import Word2Vec
import sklearn.metrics as metrics 
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

/nlp-vm/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Read in datafile

In [3]:
# Read data from files
df=pd.read_csv('subset100000.csv', header=0, sep=',', error_bad_lines=False)

In [4]:
df[0:2]

,233794,0,Sentiment140,"@joolzp1 not yet i am still searching for a good pair on line to buy lol....i just hate spending money, i dont have that much lol"
0,464813,0,Sentiment140,"@robsmada LOL!! I know, gotta let the mussels ..."
1,1497153,1,Sentiment140,morning twitter had a nice nice 10 hour sleep ...


In [6]:
df.columns = ['id', 'boolean_label', 'sentiment_source', 'abstract']

In [7]:
print len(df)
print df["abstract"].size

99999
99999


In [8]:
df[0:2]

,id,boolean_label,sentiment_source,abstract
0,464813,0,Sentiment140,"@robsmada LOL!! I know, gotta let the mussels ..."
1,1497153,1,Sentiment140,morning twitter had a nice nice 10 hour sleep ...


### Check label distribution

In [9]:
#we are concerned about rating
print len(df.loc[df['boolean_label'] == 1])
print len(df.loc[df['boolean_label'] == 0])

49972
50027


### Read in indices objects from pickle

In [10]:
trainIndices = pickle.load(open('trainIndicesTSAsub.p','rb'))
testIndices = pickle.load(open('testIndicesTSAsub.p','rb'))

In [11]:
print(len(trainIndices[0]))
print(len(testIndices))

5
25


### Apply Word2Vec Inversion to get sentence scores

In [12]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    #prob = prob.groupby("doc").mean()
    return prob

In [13]:
def getFeatures(probssentences_train, train1):
    features = np.zeros((np.unique(probssentences_train["doc"]).shape[0],11))
    label = []
    for doc in np.unique(probssentences_train["doc"]):
        inx = np.where(probssentences_train["doc"] == doc)[0]
        subset = probssentences_train[0].iloc[inx]
        score = train1['boolean_label'].iloc[doc]
        label.append(score)

        inxFirst = inx[0]
        inxLast = inx[-1]

        features[doc,0] = np.mean(subset) #mean of scores
        features[doc,1] = np.min(subset) #min score
        features[doc,2] = np.max(subset) #max score
        features[doc,3] = np.std(subset) #stdev of scores
        features[doc,4] = probssentences_train[0].iloc[inxFirst] #score of first sentence
        features[doc,5] = probssentences_train[0].iloc[inxLast] #score of last sentences
        features[doc,6] = len(subset) #number of sentences
        features[doc,7] = np.mean(                               #sigmoid (with k = 10)
                            1./(1+np.exp(-10*
                               subset.astype(float)-0.5 ))) 
        features[doc,8] = np.mean(                               #sigmoid (with k = 20)
                            1./(1+np.exp(-20*
                               subset.astype(float)-0.5 )))
        features[doc,9] = np.argmin(subset)-inxFirst #position of min score
        features[doc,10] = np.argmax(subset)-inxFirst #position of max score
    
    return (features, label)

In [14]:
def buildModels(train1, test1, docs, docsTrain, basemodel, label):
    
    scoreDT = []
    scoreRF = []
    
    # ****** Split the labeled training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    inxs_pos = np.where(train1["boolean_label"] == 1)[0].tolist()
    inxs_neg = np.where(train1["boolean_label"] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probssentencesArticle = docprobsentences(docs,models).astype(object)

    probssentences_train = docprobsentences(docsTrain,models).astype(object)
        
    #step 1- get training features
    obj = getFeatures(probssentences_train, train1)
    features = obj[0]
    label = obj[1]

    #step 2- build decision tree
    clf = tree.DecisionTreeClassifier(min_samples_leaf=1000)
    clf = clf.fit(features, label)

    print "decision tree performance on self"
    resultTrain = clf.predict(features)

    print metrics.roc_auc_score(label, resultTrain)

    #step 3- build random forest

    forest = RandomForestClassifier()
    forest = forest.fit( features, label )

    print "random forest performance on self"
    resultTrainRF = forest.predict(features)
    print metrics.roc_auc_score(label, resultTrainRF)

    #step 4- get test features
    test = getFeatures(probssentencesArticle, test1)
    featuresT = test[0]
    labelsT = test[1]

    #step 5- get test set performance on decision tree 
    result = clf.predict(featuresT)
    scoreDT.append(metrics.roc_auc_score(labelsT, result))
    scoreDT.append(metrics.accuracy_score(labelsT, result))
    scoreDT.append(metrics.f1_score(labelsT, result))
    scoreDT.append(metrics.precision_score(labelsT, result))
    scoreDT.append(metrics.recall_score(labelsT, result))
    
    print(len(scoreDT))

    #step 6- get test set performance on random forest
    clf2 = forest.predict(featuresT)
    scoreRF.append(metrics.roc_auc_score(labelsT, clf2))
    scoreRF.append(metrics.accuracy_score(labelsT, clf2))
    scoreRF.append(metrics.f1_score(labelsT, clf2))
    scoreRF.append(metrics.precision_score(labelsT, clf2))
    scoreRF.append(metrics.recall_score(labelsT, clf2))
    
    print(len(scoreDT))
    
    return scoreDT, scoreRF, models, probssentencesArticle, probssentences_train, clf, forest, features, featuresT

In [17]:
a = datetime.datetime.now().replace(microsecond=0)

#CROSS VALIDATION
#r=0-24, our meta-folds

DT_ROCs = []
DT_accuracies = []
DT_F1_scores = []
DT_precisions = []
DT_recalls = []

RF_ROCs = []
RF_accuracies = []
RF_F1_scores = []
RF_precisions = []
RF_recalls = []

for r in range(len(trainIndices)):
    
    print("METAROUND", r+1)

    m = 1

    avgROC_DT = []
    avgACC_DT = []
    avgF1_DT = []
    avgPREC_DT = []
    avgREC_DT = []
    
    avgROC_RF = []
    avgACC_RF = []
    avgF1_RF = []
    avgPREC_RF = []
    avgREC_RF = []
    
    #i=0-4, our 5-folds
    for i in range(len(trainIndices[r])):

        print("ROUND", m)
        m = m + 1
        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[trainIndices[r][i]], df.iloc[testIndices[r][i]]


        # ****** Split the labeled and unlabeled training sets into clean sentences
        #
        sentences = []  # Initialize an empty list of sentences
        docsTrain = []
        for review in train1["abstract"]:
            result = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
            sentences += result
            docsTrain.append(result)

        print "Building and training w2v models"
        ## create a w2v learner 
        basemodel = Word2Vec(
            sentences=None,
            size=100, #default
            window=5, #default
            workers=multiprocessing.cpu_count(),   # use your cores
            iter=3, # iter = sweeps of SGD through the data; more is better
            hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
        basemodel.build_vocab(sentences) 

        print "Parsing sentences from test set"
        # read in the test set as a list of a list of words
        docs = []
        for review in test1["abstract"]:
            docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

        print "Scoring test set"
        tup1 = buildModels(train1, test1, docs, docsTrain, basemodel, 1)
        scoreDT_1 = tup1[0]
        scoreRF_1 = tup1[1]

        avgROC_DT.append(scoreDT_1[0])
        avgACC_DT.append(scoreDT_1[1])
        avgF1_DT.append(scoreDT_1[2])
        avgPREC_DT.append(scoreDT_1[3])
        avgREC_DT.append(scoreDT_1[4])
        
        avgROC_RF.append(scoreRF_1[0])
        avgACC_RF.append(scoreRF_1[1])
        avgF1_RF.append(scoreRF_1[2])
        avgPREC_RF.append(scoreRF_1[3])
        avgREC_RF.append(scoreRF_1[4])
        
    print("average DT ROC:", sum(avgROC_DT)/float(len(avgROC_DT)))
    print("average DT ACC:", sum(avgACC_DT)/float(len(avgACC_DT)))
    print("average DT F1:", sum(avgF1_DT)/float(len(avgF1_DT)))
    print("average DT PREC:", sum(avgPREC_DT)/float(len(avgPREC_DT)))
    print("average DT REC:", sum(avgREC_DT)/float(len(avgREC_DT)))
    
    print("average RF ROC:", sum(avgROC_RF)/float(len(avgROC_RF)))
    print("average RF ACC:", sum(avgACC_RF)/float(len(avgACC_RF)))
    print("average RF F1:", sum(avgF1_RF)/float(len(avgF1_RF)))
    print("average RF PREC:", sum(avgPREC_RF)/float(len(avgPREC_RF)))
    print("average RF REC:", sum(avgREC_RF)/float(len(avgREC_RF)))
    
    
    DT_ROCs.append(sum(avgROC_DT)/float(len(avgROC_DT)))
    DT_accuracies.append(sum(avgACC_DT)/float(len(avgACC_DT)))
    DT_F1_scores.append(sum(avgF1_DT)/float(len(avgF1_DT)))
    DT_precisions.append(sum(avgPREC_DT)/float(len(avgPREC_DT)))
    DT_recalls.append(sum(avgREC_DT)/float(len(avgREC_DT)))
    
        
    RF_ROCs.append(sum(avgROC_RF)/float(len(avgROC_RF)))
    RF_accuracies.append(sum(avgACC_RF)/float(len(avgACC_RF)))
    RF_F1_scores.append(sum(avgF1_RF)/float(len(avgF1_RF)))
    RF_precisions.append(sum(avgPREC_RF)/float(len(avgPREC_RF)))
    RF_recalls.append(sum(avgREC_RF)/float(len(avgREC_RF)))
    
    print("--------------------------------------------------")
  
        
print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE FINAL RESULTS"
print("ROCs:")
print(DT_ROCs)
print("accuracies:")
print(DT_accuracies)
print("F1-scores:")
print(DT_F1_scores)
print("precisions:")
print(DT_precisions)
print("recalls:")
print(DT_recalls)

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST FINAL RESULTS"
print("ROCs:")
print(RF_ROCs)
print("accuracies:")
print(RF_accuracies)
print("F1-scores:")
print(RF_F1_scores)
print("precisions:")
print(RF_precisions)
print("recalls:")
print(RF_recalls)
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('METAROUND', 1)
('ROUND', 1)


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /nlp-vm/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~7beg9" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/pyth

Building and training w2v models
Parsing sentences from test set


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://twurl.cc/116o" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~73wx6" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://blip.fm/~6illw" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document 

Scoring test set
Parsing sentences from training set
scoring test set
decision tree performance on self
0.805773063458
random forest performance on self
0.962205390423
5
5
('ROUND', 2)
Building and training w2v models
Parsing sentences from test set
Scoring test set
Parsing sentences from training set
scoring test set
decision tree performance on self
0.806013662436
random forest performance on self
0.961430192054
5
5
('ROUND', 3)
Building and training w2v models
Parsing sentences from test set
Scoring test set
Parsing sentences from training set
scoring test set
decision tree performance on self
0.805293676726
random forest performance on self
0.96295721567
5
5
('ROUND', 4)
Building and training w2v models
Parsing sentences from test set
Scoring test set
Parsing sentences from training set
scoring test set
decision tree performance on self
0.803343258011
random forest performance on self
0.963531927093
5
5
('ROUND', 5)
Building and training w2v models
Parsing sentences from test set
S

In [18]:
import scipy.stats as st

print("DT Accuracy")
print(st.t.interval(0.95, len(DT_accuracies)-1, loc=np.mean(DT_accuracies), scale=st.sem(DT_accuracies)))
print("DT ROC")
print(st.t.interval(0.95, len(DT_ROCs)-1, loc=np.mean(DT_ROCs), scale=st.sem(DT_ROCs)))
print("DT F1-scores")
print(st.t.interval(0.95, len(DT_F1_scores)-1, loc=np.mean(DT_F1_scores), scale=st.sem(DT_F1_scores)))
print("DT precisions")
print(st.t.interval(0.95, len(DT_precisions)-1, loc=np.mean(DT_precisions), scale=st.sem(DT_precisions)))
print("DT recalls")
print(st.t.interval(0.95, len(DT_recalls)-1, loc=np.mean(DT_recalls), scale=st.sem(DT_recalls)))
print("--------------------------------------------")
print("RF Accuracy")
print(st.t.interval(0.95, len(RF_accuracies)-1, loc=np.mean(RF_accuracies), scale=st.sem(RF_accuracies)))
print("RF ROC")
print(st.t.interval(0.95, len(RF_ROCs)-1, loc=np.mean(RF_ROCs), scale=st.sem(RF_ROCs)))
print("RF F1-scores")
print(st.t.interval(0.95, len(RF_F1_scores)-1, loc=np.mean(RF_F1_scores), scale=st.sem(RF_F1_scores)))
print("RF precisions")
print(st.t.interval(0.95, len(RF_precisions)-1, loc=np.mean(RF_precisions), scale=st.sem(RF_precisions)))
print("RF recalls")
print(st.t.interval(0.95, len(RF_recalls)-1, loc=np.mean(RF_recalls), scale=st.sem(RF_recalls)))

DT Accuracy
(0.73671990090581474, 0.73761164886001418)
DT ROC
(0.73671667105757466, 0.73760837260823064)
DT F1-scores
(0.73482147628177152, 0.7360903746633396)
DT precisions
(0.7389272901711571, 0.74084022840696628)
DT recalls
(0.72949759147265036, 0.73299825520994644)
--------------------------------------------
RF Accuracy
(0.6982311408392331, 0.69926038541449576)
RF ROC
(0.69821922623332178, 0.69924815925675543)
RF F1-scores
(0.69138924174778771, 0.69232923445453853)
RF precisions
(0.70693689590256348, 0.70834466152260589)
RF recalls
(0.67619310195109861, 0.67738243835057077)


In [16]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences